In [14]:
%matplotlib inline
from os.path import expanduser, join
from time import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score

from sklearn.externals import joblib
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LinearRegression


## Preprare the dataset

In [50]:
%%time
# save the data into numpy version.
# note that this will take some miniutes. Be patient :)
def initialize():
    path_prefix = '/Users/Jason/Desktop/book/Learn2Rank/MSLR-WEB10K/Fold1/'
    # data_dict = {test:[x_train, y_train, qid_train]}
    data_dict = {}
    filenames = ["train.txt", "test.txt", "vali.txt"]
    for filename in filenames:
        if filename not in data_dict:
            data_dict[filename] = []
        data = pd.read_csv(path_prefix + filename, header=None, sep=' ')
        # x_train, y_train, qid_train
        rows = data.shape[0]
        columns = data.shape[1] - 1  # note that remove last ' '
        #debug
        print (filename, rows, columns)

        x_train = np.zeros((rows, columns - 2))
        y_train = np.zeros((rows, 1))
        qid_train = np.zeros((rows, 1))

        for row in range(rows):
            for column in range(columns):
                if 0 == column:
                    y_train[row] = int(data.loc[row, column])
                elif 1 == column:
                    qid_train[row] = int(data.loc[row, column].split(':')[1])
                else:
                    x_train[row][column - 2] = float(data.loc[row, column].split(':')[1])
            if not row % 10000:
                print ("process: ", str(row))
        data_dict[filename].extend([x_train, y_train, qid_train])

    np.savez(path_prefix + 'data.npz', X_train=data_dict['train.txt'][0],y_train=data_dict['train.txt'][1],qid_train=data_dict['train.txt'][2],
                X_test=data_dict['test.txt'][0], y_test=data_dict['test.txt'][1],qid_test=data_dict['test.txt'][2],
                X_vali=data_dict['vali.txt'][0], y_vali=data_dict['vali.txt'][1], qid_vali=data_dict['vali.txt'][2])

    print ("Done")
    
initialize()

train.txt 150000 138
process:  0
process:  10000
process:  20000
process:  30000
process:  40000
process:  50000
process:  60000
process:  70000
process:  80000
process:  90000
process:  100000
process:  110000
process:  120000
process:  130000
process:  140000
test.txt 50000 138
process:  0
process:  10000
process:  20000
process:  30000
process:  40000
vali.txt 50000 138
process:  0
process:  10000
process:  20000
process:  30000
process:  40000
Done
CPU times: user 6min 36s, sys: 1.57 s, total: 6min 37s
Wall time: 6min 38s


In [ ]:
%%time
# Load the numpy version data
data = np.load(expanduser('~/Desktop/book/Learn2Rank/MSLR-WEB10K/Fold1/data.npz'))
X_train, y_train, qid_train = data['X_train'], data['y_train'], data['qid_train']
X_vali, y_vali, qid_vali = data['X_vali'], data['y_vali'], data['qid_vali']
X_test, y_test, qid_test = data['X_test'], data['y_test'], data['qid_test']

In [53]:
# Get size in bytes
print ((X_train.nbytes + X_vali.nbytes + X_test.nbytes) / 1e6)
# Get number of search results
print (len(X_train) + len(X_vali) + len(X_test))
# Get number of queries
print (len(np.unique(qid_train)) + len(np.unique(qid_vali)) + len(np.unique(qid_test)))

272.0
250000
2137


In [54]:
# Concatenate the training and validation sets as a big development set.
X_dev = np.vstack([X_train, X_vali])
y_dev = np.concatenate([y_train, y_vali])
qid_dev = np.concatenate([qid_train, qid_vali])

In [55]:
print (X_dev.shape)
print (X_dev.dtype)

(200000, 136)
float64


In [58]:
# Extract a subset of 500 queries to speed up learning
def subsample(X, y, qid, size, seed=None):
    rng = np.random.RandomState(seed)
    unique_qid = np.unique(qid)
    qid_mask = rng.permutation(len(unique_qid))[:size]
    subset_mask = np.in1d(qid, unique_qid[qid_mask])
    return X[subset_mask], y[subset_mask], qid[subset_mask]

X_train_small, y_train_small, qid_train_small = subsample(
    X_train, y_train, qid_train, 500, seed=0)

In [60]:
## Double Check data
print (X_train_small.shape)
print (len(np.unique(qid_train_small)))

(63148, 136)
500


In [61]:
# adjust training data to be balanced
def balance_irrelevant(X, y, qid, seed=None):
    """Subsample the zero-scored entries"""
    rng = np.random.RandomState(seed)
    unique_qid = np.unique(qid)
    final_mask = np.ones(shape=y.shape, dtype=np.bool)
    for this_qid in unique_qid:
        this_mask = qid == this_qid
        this_y = y[this_mask]
        relevant = this_y >= 2
        ratio = float(np.mean(relevant))
        if ratio > 0.5:
            # already balanced
            continue
            
        final_mask[this_mask] = np.logical_or(
            relevant, np.random.random(len(this_y)) > 0.7) 
    return X[final_mask], y[final_mask], qid[final_mask]


X_train_medium, y_train_medium, qid_train_medium = subsample(
    X_train, y_train, qid_train, 1000, seed=0)

# Get balanced sample data
X_balanced_small, y_balanced_small, qid_balanced_small = balance_irrelevant(
    X_train_small, y_train_small, qid_train_small)

print(len(y_train_small))
print(len(y_balanced_small))

63148
25606


/Users/Jason/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:18: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 1; dimension is 136 but corresponding boolean dimension is 1


## Evaluate ranking with NDCG


In [63]:
def dcg(relevances, rank=10):
    """Discounted cumulative gain at rank (DCG)"""
    relevances = np.asarray(relevances)[:rank]
    n_relevances = len(relevances)
    if n_relevances == 0:
        return 0.

    discounts = np.log2(np.arange(n_relevances) + 2)
    return np.sum(relevances / discounts)
 
 
def ndcg(relevances, rank=10):
    """Normalized discounted cumulative gain (NDGC)"""
    best_dcg = dcg(sorted(relevances, reverse=True), rank)
    if best_dcg == 0:
        return 0.

    return dcg(relevances, rank) / best_dcg

In [64]:
ndcg([2, 4, 0, 1, 1, 0, 0], rank=5)

0.86253003992915656

In [65]:
ndcg([0, 0, 0, 1, 1, 2, 4], rank=5)

0.13201850690866795

In [66]:
ndcg([0, 0, 0, 1, 1, 2, 4], rank=3)

0.0

In [67]:
ndcg([4, 2, 1, 1, 0, 0, 0], rank=5)

1.0

In [68]:
def mean_ndcg(y_true, y_pred, query_ids, rank=10):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    query_ids = np.asarray(query_ids)
    # assume query_ids are sorted
    ndcg_scores = []
    previous_qid = query_ids[0]
    previous_loc = 0
    for loc, qid in enumerate(query_ids):
        if previous_qid != qid:
            chunk = slice(previous_loc, loc)
            ranked_relevances = y_true[chunk][np.argsort(y_pred[chunk])[::-1]]
            ndcg_scores.append(ndcg(ranked_relevances, rank=rank))
            previous_loc = loc
        previous_qid = qid

    chunk = slice(previous_loc, loc + 1)
    ranked_relevances = y_true[chunk][np.argsort(y_pred[chunk])[::-1]]
    ndcg_scores.append(ndcg(ranked_relevances, rank=rank))
    return np.mean(ndcg_scores)


mean_ndcg([4, 3, 1, 4, 3], [4, 0, 1, 4, 2], [0, 0, 0, 2, 2], rank=10)

0.9795191506818377

In [69]:
def print_evaluation(model, X, y, qid):
    tic = time()
    y_predicted = model.predict(X)
    prediction_time = time() - tic
    print("Prediction time: {:.3f}s".format(prediction_time))
    print("NDCG@5 score: {:.3f}".format(
    mean_ndcg(y, y_predicted, qid, rank=5)))
    print("NDCG@10 score: {:.3f}".format(
    mean_ndcg(y, y_predicted, qid, rank=10)))
    print("NDCG score: {:.3f}".format(
    mean_ndcg(y, y_predicted, qid, rank=None)))
    print("R2 score: {:.3f}".format(r2_score(y, y_predicted)))


## Try to adopt different ensemble models

In [70]:
%%time

from sklearn.ensemble import ExtraTreesRegressor

etr = ExtraTreesRegressor(n_estimators=200, min_samples_split=5, random_state=1, n_jobs=-1)
etr.fit(X_train_small, y_train_small)

/Users/Jason/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


CPU times: user 9min 24s, sys: 3.79 s, total: 9min 28s
Wall time: 2min 44s


In [71]:
print_evaluation(etr, X_test, y_test, qid_test)

Prediction time: 1.363s
NDCG@5 score: 0.506
NDCG@10 score: 0.521
NDCG score: 0.985
R2 score: 0.167


In [74]:
%%time

from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor(n_estimators=200, random_state=1, verbose=1)
gbr.fit(X_train_small, y_train_small)

/Users/Jason/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


      Iter       Train Loss   Remaining Time 
         1           0.6364            2.57m
         2           0.6252            2.43m
         3           0.6159            2.47m
         4           0.6074            2.51m
         5           0.6009            2.45m
         6           0.5949            2.41m
         7           0.5896            2.39m
         8           0.5851            2.36m
         9           0.5811            2.34m
        10           0.5776            2.40m
        20           0.5557            2.28m
        30           0.5433            2.19m
        40           0.5362            2.03m
        50           0.5303            1.95m
        60           0.5262            1.83m
        70           0.5228            1.72m
        80           0.5196            1.57m
        90           0.5165            1.44m
       100           0.5139            1.30m
       200           0.4925            0.00s
CPU times: user 2min 32s, sys: 1.21 s, total: 2min 34s

In [75]:
print_evaluation(gbr, X_test, y_test, qid_test)

Prediction time: 0.421s
NDCG@5 score: 0.529
NDCG@10 score: 0.546
NDCG score: 0.985
R2 score: 0.185


In [79]:
%%time

from sklearn.ensemble import GradientBoostingRegressor

gbr2 = GradientBoostingRegressor(n_estimators=200, max_depth=3,
                                 learning_rate=0.1, loss='ls',
                                 random_state=1, verbose=1)
gbr2.fit(X_dev, y_dev)

/Users/Jason/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


      Iter       Train Loss   Remaining Time 
         1           0.6357            9.98m
         2           0.6242            9.55m
         3           0.6147            9.40m
         4           0.6063            9.30m
         5           0.5992            9.23m
         6           0.5933            9.15m
         7           0.5883            9.05m
         8           0.5838            9.05m
         9           0.5799            9.02m
        10           0.5765            8.97m
        20           0.5560            8.47m
        30           0.5457            7.94m
        40           0.5393            7.48m
        50           0.5351            6.97m
        60           0.5319            6.47m
        70           0.5292            5.99m
        80           0.5268            5.50m
        90           0.5252            5.01m
       100           0.5236            4.54m
       200           0.5115            0.00s
CPU times: user 9min 5s, sys: 2.07 s, total: 9min 7s
W

In [80]:
print_evaluation(gbr2, X_test, y_test, qid_test)

Prediction time: 0.483s
NDCG@5 score: 0.536
NDCG@10 score: 0.557
NDCG score: 0.985
R2 score: 0.198


## Comparing with a baseline linear regression models (with different optimizers and input scaling)

In [81]:
%time lr = LinearRegression().fit(X_dev, y_dev)

CPU times: user 2.65 s, sys: 485 ms, total: 3.14 s
Wall time: 2.36 s


In [82]:
print_evaluation(lr, X_test, y_test, qid_test)

Prediction time: 0.012s
NDCG@5 score: 0.375
NDCG@10 score: 0.375
NDCG score: 0.375
R2 score: 0.147


#### Evaluate overfitting by comparing with training set:

In [83]:
print_evaluation(lr, X_dev, y_dev, qid_dev)

Prediction time: 0.036s
NDCG@5 score: 0.393
NDCG@10 score: 0.393
NDCG score: 0.393
R2 score: 0.141


#### Interestingly enough, a slight overfitting of the training set from a regression standpoint (higher r2 score) does not seem to cause overfitting from a ranking standpoint. This would have to be checked with cross-validation though.

#### Let's evaluate the impact of imput feature normalization:

In [84]:
%%time

nlr = LinearRegression(normalize=True).fit(X_dev, y_dev)

CPU times: user 2.63 s, sys: 363 ms, total: 3 s
Wall time: 2.16 s


In [85]:
print_evaluation(nlr, X_test, y_test, qid_test)

Prediction time: 0.010s
NDCG@5 score: 0.375
NDCG@10 score: 0.375
NDCG score: 0.375
R2 score: 0.147


In [86]:
%%time

nlr = LinearRegression(normalize=True).fit(X_train_small, y_train_small)

CPU times: user 772 ms, sys: 116 ms, total: 888 ms
Wall time: 656 ms


In [87]:
print_evaluation(nlr, X_test, y_test, qid_test)

Prediction time: 0.010s
NDCG@5 score: 0.375
NDCG@10 score: 0.375
NDCG score: 0.375
R2 score: 0.065


In [88]:
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_small_scaled = scaler.fit_transform(X_train_small)
X_test_scaled = scaler.transform(X_test)

In [89]:
%%time

sgdlr = SGDRegressor(alpha=1e-7, learning_rate='constant', eta0=1e-5, n_iter=50).fit(X_train_small_scaled, y_train_small)

/Users/Jason/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:73: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/Users/Jason/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


CPU times: user 1.71 s, sys: 22.9 ms, total: 1.73 s
Wall time: 1.82 s


In [90]:
print_evaluation(sgdlr, X_test_scaled, y_test, qid_test)

Prediction time: 0.012s
NDCG@5 score: 0.466
NDCG@10 score: 0.496
NDCG score: 0.985
R2 score: 0.140


## Comparing with a classification to NDCG ranking reduction models


In [91]:
from sklearn.linear_model import LogisticRegression
from sklearn.base import RegressorMixin
from sklearn.preprocessing import StandardScaler
from sklearn.base import clone

In [92]:
def proba_to_relevance(probas):
    """MCRank-like reduction of classification proba to DCG predictions"""
    rel = np.zeros(probas.shape[0], dtype=np.float32)
    for i in range(probas.shape[1]):
        rel += i * probas[:, i]
    return rel
        
        
class ClassificationRanker(RegressorMixin):
    
    def __init__(self, base_estimator=None):
        self.base_estimator = base_estimator
        
    def fit(self, X, y):
        self.estimator_ = clone(self.base_estimator)
        self.scaler_ = StandardScaler()
        X = self.scaler_.fit_transform(X)
        self.estimator_.fit(X, y)
        
    def predict(self, X):
        X_scaled = self.scaler_.transform(X)
        probas = self.estimator_.predict_proba(X_scaled)
        return proba_to_relevance(probas)

In [93]:
from sklearn.linear_model import SGDClassifier

sgdlogr = SGDClassifier(loss='modified_huber', alpha=1e-8, n_iter=200, learning_rate='constant', eta0=1e-6, n_jobs=-1)
sgdlogrr = ClassificationRanker(sgdlogr)
sgdlogrr.fit(X_train_small_scaled, y_train_small)

/Users/Jason/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:73: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/Users/Jason/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [94]:
print_evaluation(sgdlogrr, X_test_scaled, y_test, qid_test)

Prediction time: 0.128s
NDCG@5 score: 0.468
NDCG@10 score: 0.500
NDCG score: 0.985
R2 score: 0.147


In [95]:
%%time

from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=200, max_features=20, min_samples_split=5,
                            random_state=1, n_jobs=-1)
rfr = ClassificationRanker(rfc)
rfr.fit(X_train_small, y_train_small)

/Users/Jason/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


CPU times: user 3min 33s, sys: 1.42 s, total: 3min 34s
Wall time: 59.1 s


In [96]:
print_evaluation(rfr, X_test, y_test, qid_test)

Prediction time: 2.186s
NDCG@5 score: 0.506
NDCG@10 score: 0.520
NDCG score: 0.985
R2 score: 0.165


In [97]:
%%time

from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(n_estimators=100, random_state=1)
gbr = ClassificationRanker(gbc)
gbr.fit(X_train_small, y_train_small)

/Users/Jason/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


CPU times: user 5min 55s, sys: 2.8 s, total: 5min 58s
Wall time: 6min 3s


In [98]:
print_evaluation(gbr, X_test, y_test, qid_test)

Prediction time: 1.096s
NDCG@5 score: 0.529
NDCG@10 score: 0.539
NDCG score: 0.985
R2 score: 0.174


In [99]:
%%time

from sklearn.ensemble import ExtraTreesClassifier

etc = ClassificationRanker(ExtraTreesClassifier(n_estimators=200, random_state=1, n_jobs=-1))
etc.fit(X_train_small, y_train_small)

/Users/Jason/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


CPU times: user 1min 12s, sys: 2.18 s, total: 1min 15s
Wall time: 23.6 s


In [100]:
print_evaluation(etc, X_test, y_test, qid_test)

Prediction time: 3.186s
NDCG@5 score: 0.471
NDCG@10 score: 0.490
NDCG score: 0.985
R2 score: 0.150
